In [1]:
import os
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from Agent import DQNAgent, preprocess

In [1]:
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
log_dir = os.path.abspath("../logs")

In [4]:
EPISODES = 2
render = True

In [5]:
# Exploration settings
epsilon = 0.2  # not a constant, going to be decayed
EPSILON_DECAY = 0.999
MIN_EPSILON = 0.001

In [6]:
# Stats settings
AGGREGATE_STATS_EVERY = 50  # episodes
SHOW_PREVIEW = False

In [7]:
env = gym.make("MountainCar-v0")
env.reset()

print(f"Observation high values: {env.observation_space.high}")
print(f"Observation low values: {env.observation_space.low}")
print(f"Number of actions: {env.action_space.n}")

Observation high values: [0.6  0.07]
Observation low values: [-1.2  -0.07]
Number of actions: 3


In [8]:
agent = DQNAgent(env, log_dir)

Starting log in /home/claus/git/AI/gym/logs/Dense8-1565193347


In [9]:
ep_rewards = []
epsilons = []

for episode in tqdm(range(EPISODES), ncols=1000):
    episode_reward = 0

    # Update tensorboard step every episode
    agent.tensorboard.step = episode

    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1

    # Reset environment and get initial state
    current_state = preprocess(env.reset())

    # Reset flag and start iterating until episode ends
    done = False
    while not done:

        # This part stays mostly the same, the change is to query a model for Q values
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)

        new_state, reward, done, _ = env.step(action)
        new_state = preprocess(new_state)

        # Transform new continous state to new discrete state and count reward
        episode_reward += reward

        if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:
            env.render()

        # Every step we update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)

        current_state = new_state
        step += 1

    ep_rewards.append(episode_reward)

    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

    epsilons.append(epsilon)

fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].plot(ep_rewards)
ax[0].set_title("Rewards")
ax[1].plot(epsilons)
ax[1].set_title("Epsilon")
plt.show()

AttributeError: module 'tensorflow' has no attribute 'Summary'